In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [2]:
from lightgbm import LGBMClassifier

In [3]:
import seaborn as sns
sns.set()

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [71]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
# train data 입력시 id, target 모두 넣어주어야 한다. 
# test data 입력시 id 모두 넣어주어야 한다.
# from sklearn.preprocessing import LabelEncoder 해주어야 함
def train_model(df_train, df_test, model, path, is_submit = True, is_score = True, user_id = "id", target = "country_destination"):
    le = LabelEncoder()

    # feature, target 전처리
    if is_submit == True:
        y_train = le.fit_transform(df_train[target])
    else:
        y_train = df_train[target]
    X_train = df_train.drop([target, user_id], axis = 1)

    X_test_id = df_test[user_id] # id 따로 정의
    X_test = df_test.drop([user_id], axis = 1)

    # train data set으로 fitting
    print("model fitting 시작 !!")
    model = model.fit(X_train, y_train)
    predic_proba = model.predict_proba(X_test)
    print("model fitting 완료 !!")
    
    if is_submit == True:
        # submit data, predict data 생성
        df_submit = pd.DataFrame(columns=["id", "country"])

        ids = []
        cts = []
        for i in range(len(X_test_id)):
            idx = X_test_id.iloc[i]
            ids += [idx] * 5
            cts += le.inverse_transform(np.argsort(predic_proba[i])[::-1])[:5].tolist()

        df_submit["id"] = ids
        df_submit["country"] = cts

        df_submit.to_csv(path, index = False)
        print("csv file 제작 완료 !!")

        !kaggle competitions submit -c airbnb-recruiting-new-user-bookings -f {path} -m "Message"
    
    if is_score == True:
        print("cross valiation 시작 !!")
        print(cross_val_score(model, X_train, y_train, scoring = "neg_log_loss", cv = 3, n_jobs = 4))
        print("cross valiation 끝 !!")
    
    return model

In [23]:
df_train = pd.read_csv("../data/train_age_action_count_lag.csv")
df_test = pd.read_csv("../data/test_age_action_count_lag.csv")

In [44]:
df_train_origin = pd.read_csv("../data/train_users_2.csv")
df_test_origin = pd.read_csv("../data/test_users.csv")

### Make 무성의로 사인한 사람들

In [72]:
s_all_input_train = np.logical_not(df_train_origin['age'] < 120) | (df_train_origin['gender'] == '-unknown-')
s_all_input_test = np.logical_not(df_test_origin['age'] < 120) | (df_test_origin['gender'] == '-unknown-')

In [46]:
df_train['all_input_sign'] = s_all_input_train.apply(lambda x : 1 if x == True else 0)
df_test['all_input_sign'] = s_all_input_test.apply(lambda x : 1 if x == True else 0)

In [48]:
df_train.shape, df_test.shape

((213451, 521), (62096, 520))

In [62]:
lgb = LGBMClassifier(max_depth = 4, n_jobs=4)

In [73]:
train_model(df_train, df_test, lgb, "submit/submit.csv", is_submit = False, is_score=False)

model fitting 시작 !!
model fitting 완료 !!


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=4, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=4, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1)

In [74]:
lgb.classes_

array(['AU', 'CA', 'DE', 'ES', 'FR', 'GB', 'IT', 'NDF', 'NL', 'PT', 'US',
       'other'], dtype=object)

In [78]:
y = df_train["country_destination"]
y_pre = lgb.predict(df_train.drop(["id", "country_destination"], axis = 1))

### log_loss

In [88]:
y_train = df_train["country_destination"]
X_train = df_train.drop(["id", "country_destination"], axis = 1)

In [ ]:
cross_val_score(lgb, X_train, y_train, scoring = "neg_log_loss", cv = 3, n_jobs = 4)

### confusion matrix

In [86]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [87]:
print(confusion_matrix(y,y_pre))
print(classification_report(y,y_pre))

[[     0      0      0      0      0      0      0    292      0      0
     247      0]
 [     0      4      0      0      0      0      0    737      0      0
     687      0]
 [     0      0      4      0      0      0      0    588      0      0
     469      0]
 [     0      0      0      0      0      0      0   1259      0      0
     990      0]
 [     0      0      0      0     18      0      1   2825      0      0
    2178      1]
 [     0      0      0      0      1      1      0   1271      0      0
    1051      0]
 [     0      0      0      0      1      0     11   1606      0      0
    1216      1]
 [     0      0      0      0      1      0      0 108857      0      0
   15685      0]
 [     0      0      0      0      0      0      0    409      3      0
     350      0]
 [     0      0      0      0      0      0      0    115      0      7
      95      0]
 [     0      0      0      0      0      0      0  33279      0      0
   29097      0]
 [     0      0      